In [2]:
import time
import datetime 
from pprint import pprint
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [38]:
# Step1: 爬取資料 取至前5000名，並去除欄位名
top_games_all = []
game_ids = []
r = 1
while True:
    url = f'https://steamcharts.com/top/p.{r}'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        top = soup.find('table',id="top-games", class_='common-table')
        _id = top.find_all('a')
        top_games = top.get_text()
        top_games = re.split(r'[\n\t]', top_games)
        top_games.remove('Name')
        top_games.remove('Current Players')
        top_games.remove('Last 30 Days')
        top_games.remove('Peak Players')
        top_games.remove('Hours Played')
        
        for s in top_games:
            if s != '':
                top_games_all.append(s)

        for link in _id:
            href = link.get('href')
            href = href.split('/app/')[1]
            game_ids.append(href)     
        r += 1
    else:
        break
# Step2: 將取出的資料進行，每5個分組
top_new = []
chunk_size = 5
for i in range(0, len(top_games_all), chunk_size):
    chunk = top_games_all[i:i+chunk_size]
    top_new.append(chunk)
top_new

# Step3: 將資料轉成 DataFrame
columns = ['Rank', 'Name', 'Current_Players', 'Peak_Players', 'Hours_Played(30days)']
data = []
for r in top_new:
    data.append(r)

df = pd.DataFrame(
    data=data,
    columns=columns
)
df.insert(loc=1, column='App_Id', value=game_ids)
df = df.drop('Rank', axis=1)
df = df.drop('Current_Players', axis=1)
df = df.drop('Peak_Players', axis=1)
df = df.drop('Hours_Played(30days)', axis=1)
df
#df.to_csv('Steam_id.csv', encoding='utf-8', index=False)

,App_Id,Name
0,730,Counter-Strike 2
1,2923300,Banana
2,570,Dota 2
3,578080,PUBG: BATTLEGROUNDS
4,2139460,Once Human
...,...,...
13450,1447260,Mystical Echoes
13451,572940,Bungo Stray Dogs: Fortune is Unpredictable and...
13452,1621110,Tabletop Simulator - Adventure Mart
13453,1519240,Box Ninja
